In [1]:
import lightgbm as lgb, catboost as cat

In [2]:
import pandas as pd
import joblib

In [3]:
import numpy as np
import random

In [4]:
np.random.seed(42)
random.seed(42)

In [5]:
def read_merge_data(data_type="training"):
    #
    Dfs = []
    train_df = []
    train_output = pd.read_csv("./data_phase_one/training_output.csv", header=None)
    print("Train output:", train_output.shape)
    coords = pd.read_csv("data_phase_one/coords.csv", header=None)
    for name in ["mslp", "sst", "ssta", "t2m"]:
        df = pd.read_csv(f"./data_phase_one/{data_type}_input_{name}.csv", header=None)
        print("Shape df", df.shape)
        Dfs.append(df)
    j = 0
    for i in range(0, df.shape[0]-12, 1):
        feat = pd.concat([coords, Dfs[0].iloc[i:i+12], Dfs[1].iloc[i:i+12], Dfs[2].iloc[i:i+12], Dfs[3].iloc[i:i+12]], axis=0).reset_index(drop=True)
        tgt = train_output.iloc[i+11]
        tgt = tgt.T.reset_index(drop=True)
        feat = feat.T.reset_index(drop=True)
        feat = pd.concat([feat, tgt], axis=1)
        assert feat.shape[1]==51
        feat.columns = [f"feat_{k}" for k in range(feat.shape[1]-1)] + ["target"]
        if j < 3:
            print(feat.columns)
        train_df.append(feat)
        j += 1
    train_df = pd.concat(train_df, axis=0)
    #Dfs.index = [f"col_{i}" for i in range(848)]
    return train_df

In [6]:
#pd.read_csv("data_phase_one/coords.csv", header=None)

In [7]:
data = read_merge_data()

Train output: (849, 5774)
Shape df (849, 5774)
Shape df (849, 5774)
Shape df (849, 5774)
Shape df (849, 5774)
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat_21', 'feat_22', 'feat_23', 'feat_24',
       'feat_25', 'feat_26', 'feat_27', 'feat_28', 'feat_29', 'feat_30',
       'feat_31', 'feat_32', 'feat_33', 'feat_34', 'feat_35', 'feat_36',
       'feat_37', 'feat_38', 'feat_39', 'feat_40', 'feat_41', 'feat_42',
       'feat_43', 'feat_44', 'feat_45', 'feat_46', 'feat_47', 'feat_48',
       'feat_49', 'target'],
      dtype='object')
Index(['feat_0', 'feat_1', 'feat_2', 'feat_3', 'feat_4', 'feat_5', 'feat_6',
       'feat_7', 'feat_8', 'feat_9', 'feat_10', 'feat_11', 'feat_12',
       'feat_13', 'feat_14', 'feat_15', 'feat_16', 'feat_17', 'feat_18',
       'feat_19', 'feat_20', 'feat

In [8]:
data.shape

(4832838, 51)

In [9]:
data

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,...,feat_41,feat_42,feat_43,feat_44,feat_45,feat_46,feat_47,feat_48,feat_49,target
0,-64.0,-180.0000,98848.380,99195.690,99320.914,98469.160,98149.220,98269.195,98373.570,97659.090,...,272.15683,270.06723,268.31836,263.92892,263.46680,266.21063,266.94644,269.52716,272.84732,0.508057
1,-64.0,-177.1875,98898.160,99224.910,99322.336,98480.360,98138.870,98256.570,98282.520,97621.800,...,272.32718,270.27383,268.16794,264.03406,263.46497,265.92792,267.64600,269.82437,272.84190,0.895020
2,-64.0,-174.3750,98945.240,99245.610,99325.310,98495.400,98137.020,98236.720,98203.670,97581.520,...,272.27463,270.41702,268.08820,264.11017,264.33670,265.56000,267.92688,269.89145,272.67154,0.235443
3,-64.0,-171.5625,98972.610,99259.370,99331.414,98527.730,98152.336,98207.220,98140.990,97533.445,...,272.41055,270.53120,268.10270,264.25516,264.56506,265.06345,268.00845,270.06723,272.62080,0.018951
4,-64.0,-168.7500,98969.490,99280.640,99335.805,98585.020,98189.350,98166.090,98090.790,97491.330,...,272.36887,270.52940,268.13530,264.38925,263.41425,264.56143,268.06280,270.17780,272.54285,-0.058502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5769,62.0,-5.6250,100949.125,101105.414,99110.170,100612.875,101548.020,100916.230,101062.870,101320.266,...,279.02730,278.56155,276.80722,278.99832,278.89682,279.85193,282.33840,283.98760,284.57117,0.470673
5770,62.0,-2.8125,101014.790,101204.120,99232.990,100601.810,101610.990,100828.586,101021.590,101323.670,...,279.06717,278.43470,276.64410,279.00012,279.24840,280.11652,282.72443,284.54760,285.18555,0.567230
5771,62.0,0.0000,101065.984,101295.586,99371.120,100607.914,101676.370,100745.620,100973.234,101342.250,...,279.12880,278.28064,276.68400,279.06357,279.47678,280.28506,283.03976,285.11487,285.78180,0.351593
5772,62.0,2.8125,101128.520,101377.560,99537.330,100660.810,101771.670,100720.375,100953.950,101417.695,...,278.78445,277.08087,276.23636,278.19003,278.98200,280.00595,283.07060,285.47010,286.57560,-0.421692


In [10]:
data.iloc[:,:-1].values.reshape(-1, 5774)

array([[-6.40000000e+01, -1.80000000e+02,  9.88483800e+04, ...,
         2.71460970e+02,  2.71459750e+02,  2.71460360e+02],
       [ 2.71459750e+02,  2.71623100e+02, -2.84637450e-01, ...,
         2.68245850e+02,  2.67211030e+02,  2.67908780e+02],
       [ 2.70795780e+02,  2.72377930e+02, -6.20000000e+01, ...,
         2.71671400e+02,  2.71462220e+02,  2.71460970e+02],
       ...,
       [-1.08923340e+00, -1.24667360e+00, -9.48028560e-01, ...,
         2.85537140e+02,  5.20000000e+01,  1.51875000e+02],
       [ 1.01275734e+05,  1.00817945e+05,  1.01120870e+05, ...,
         2.76048250e+02,  2.79330440e+02,  2.81318050e+02],
       [-1.96228030e-01, -4.06219480e-01, -9.40155000e-01, ...,
         2.83092350e+02,  2.90281770e+02,  2.88947900e+02]])

In [11]:
data.shape

(4832838, 51)

In [12]:
#y = train_df["target"]

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [14]:
cv = KFold(n_splits=5, shuffle=True)

In [15]:
params_lgb = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "max_depth": 8,
    'num_leaves': 255,
    "learning_rate": 0.01,
    "n_estimators": 2000,
    #"colsample_bytree": 0.8, 
    #"colsample_bynode": 0.8,
    "verbose": -1,
    "device": "cpu",
    #"gpu_device_id": 1
}

In [16]:
params_catb = dict(
task_type="CPU",
reg_lambda=0.438, learning_rate=0.1,
max_depth=8, min_data_in_leaf=50,
n_estimators=2000, verbose=50,
objective='RMSE')

In [17]:
params_catb

{'task_type': 'CPU',
 'devices': '1',
 'reg_lambda': 0.438,
 'learning_rate': 0.1,
 'max_depth': 8,
 'min_data_in_leaf': 50,
 'n_estimators': 2000,
 'verbose': 50,
 'objective': 'RMSE'}

In [18]:
for state, (train_idx, val_idx) in enumerate(cv.split(data)):
    X_train = data.drop(columns=["target"]).iloc[train_idx].values
    X_val = data.drop(columns=["target"]).iloc[val_idx].values
    y_train = data["target"].iloc[train_idx].values
    y_val = data["target"].iloc[val_idx].values
    if state==3:
        break
    print(f"\nTrain: {len(y_train)}, Val: {len(y_val)}\n")
    lgb_model = lgb.LGBMRegressor(**params_lgb, random_state=state)
    lgb_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)])
    joblib.dump(lgb_model, f"./kfold_trained2/lgb_model_fold{state}.joblib")

In [19]:
for state, (train_idx, val_idx) in enumerate(cv.split(data)):
    X_train = data.drop(columns=["target"]).iloc[train_idx].values
    X_val = data.drop(columns=["target"]).iloc[val_idx].values
    y_train = data["target"].iloc[train_idx].values
    y_val = data["target"].iloc[val_idx].values
    print(f"\nTrain: {len(y_train)}, Val: {len(y_val)}\n")
    if state==3:
        break
    catb_model = cat.CatBoostRegressor(**params_catb, random_state=state)
    catb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=200)
    joblib.dump(catb_model, f"./kfold_trained2/catb_model_fold{state}.joblib")


Train: 3866270, Val: 966568


Train: 3866270, Val: 966568


Train: 3866270, Val: 966568


Train: 3866271, Val: 966567


Train: 3866271, Val: 966567

